In [ ]:
# Install required libraries (run this once)
!pip install -q transformers accelerate einops bitsandbytes --break-system-packages --no-deps

In [ ]:
# Load prompts
import json

prompt_path = "/kaggle/input/new-prompts/dynamic_prompts.json"
with open(prompt_path) as f:
    prompts = json.load(f)

In [ ]:
# Format prompt
def format_prompt(prompt):
    return (
        "<|im_start|>system\n"
        "You are a C code generator.\n"
        "Output **only** a single, valid, compilable C program-nothing else.\n"
        "DO NOT include any explanations, examples, extra text, or comments.\n"
        "<|im_end|>\n"
        "<|im_start|>user\n"
        f"{prompt.strip()}\n"
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

In [ ]:
# Inference function
import torch

@torch.inference_mode()
def generate_response(prompt_text, max_tokens=512):
    # tokenise once
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
    input_ids = inputs.input_ids

    with torch.no_grad():
        # this returns [1, prompt_len + gen_len]
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # slice off the prompt tokens, keep only newly generated ones
    gen_ids = output_ids[0, input_ids.shape[1]:]

    # decode *only* the new tokens
    generated = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return generated

In [ ]:
# Load model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "Qwen/Qwen2.5-Coder-7B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

In [ ]:
#Run and save results
from tqdm import tqdm
import os

results = []

for prompt_obj in tqdm(prompts):
   prompt_text = format_prompt(prompt_obj["prompt"])
   try:
       output = generate_response(prompt_text)
   except Exception as e:
       output = f"ERROR: {e}"

   prompt_obj["model"] = "Qwen2.5-Coder-7B-Instruct"
   prompt_obj["input"] = prompt_text
   prompt_obj["output"] = output
   results.append(prompt_obj)

os.makedirs("results", exist_ok=True)
with open("results/qwen2_results.json", "w") as f:
   json.dump(results, f, indent=2)

print(f"Saved {len(results)} results to results/qwen2_results.json")